<a href="https://colab.research.google.com/github/shashi3876/kaggle/blob/main/PodcastListenPredictLightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

THis notebook contains the work done for the Kaggel playgorund competition named Predict Podcast Listening Time. Mainly I'm using LIghtGBM in this notebook for the predcitions.

In [10]:
# imports

import pandas as pd

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [18]:
train.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,63.84,True Crime,74.81,Thursday,Night,53.58,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


# Preprocess Data

First I'm filling the null values of train and test set.

In [14]:
train.isna().sum()

,0
id,0
Podcast_Name,0
Episode_Title,0
Episode_Length_minutes,87093
Genre,0
Host_Popularity_percentage,0
Publication_Day,0
Publication_Time,0
Guest_Popularity_percentage,146030
Number_of_Ads,1


In [17]:
train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
test['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)

train['Number_of_Ads'].fillna(train['Number_of_Ads'].median(), inplace=True)
train['Guest_Popularity_percentage'].fillna(train['Guest_Popularity_percentage'].median(), inplace=True)


test['Guest_Popularity_percentage'].fillna(test['Guest_Popularity_percentage'].median(), inplace=True)

<ipython-input-17-099d9f371e57>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
<ipython-input-17-099d9f371e57>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

Now doing the label encoding on the categorical data

In [20]:
categorical_features = [
    'Podcast_Name',
    'Genre',
    'Publication_Day',
    'Publication_Time',
    'Episode_Sentiment'
]
from sklearn.preprocessing import LabelEncoder

# Encoder for categorical data
label_encoders = {col: LabelEncoder() for col in categorical_features}

# Apply LabelEncoder to each categorical column
for col in categorical_features:
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

    # Converting to category type
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [23]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title'])
test = test.drop(columns=['Episode_Title'])

In [24]:
train.head()

,id,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num
0,0,34,63.84,9,74.81,4,3,53.58,0.0,2,31.41998,98
1,1,24,119.80,1,66.95,2,0,75.95,2.0,0,88.01241,26
2,2,40,73.90,2,69.97,5,1,8.97,0.0,0,44.92531,16
3,3,10,67.17,8,57.22,1,2,78.70,2.0,2,46.27824,45
4,4,31,110.51,3,80.07,1,0,58.68,3.0,1,75.61031,86


In [30]:
train.columns

Index(['id', 'Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num'],
      dtype='object')

In [ ]:
categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']

# Build the model

In [25]:
X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

In [31]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=1000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)

<ipython-input-31-7801a0c50960>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-31-7801a0c50960>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2012
[200]	valid_0's rmse: 12.9311
[300]	valid_0's rmse: 12.9061
[400]	valid_0's rmse: 12.8964
[500]	valid_0's rmse: 12.891
[600]	valid_0's rmse: 12.8873
[700]	valid_0's rmse: 12.8852
[800]	valid_0's rmse: 12.8818
[900]	valid_0's rmse: 12.8769
[1000]	valid_0's rmse: 12.8738
Did not meet early stopping. Best iteration is:
[967]	valid_0's rmse: 12.8735


<ipython-input-31-7801a0c50960>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-31-7801a0c50960>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2451
[200]	valid_0's rmse: 12.9745
[300]	valid_0's rmse: 12.9527
[400]	valid_0's rmse: 12.9443
[500]	valid_0's rmse: 12.9367
[600]	valid_0's rmse: 12.9335
[700]	valid_0's rmse: 12.9289
[800]	valid_0's rmse: 12.9276
[900]	valid_0's rmse: 12.9255
[1000]	valid_0's rmse: 12.9231
Did not meet early stopping. Best iteration is:
[982]	valid_0's rmse: 12.9229


<ipython-input-31-7801a0c50960>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-31-7801a0c50960>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.239
[200]	valid_0's rmse: 12.9684
[300]	valid_0's rmse: 12.942
[400]	valid_0's rmse: 12.9316
[500]	valid_0's rmse: 12.9274
[600]	valid_0's rmse: 12.9232
[700]	valid_0's rmse: 12.9215
[800]	valid_0's rmse: 12.918
[900]	valid_0's rmse: 12.9132
[1000]	valid_0's rmse: 12.91
Did not meet early stopping. Best iteration is:
[989]	valid_0's rmse: 12.9096


<ipython-input-31-7801a0c50960>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-31-7801a0c50960>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2526
[200]	valid_0's rmse: 12.972
[300]	valid_0's rmse: 12.9489
[400]	valid_0's rmse: 12.9382
[500]	valid_0's rmse: 12.9316
[600]	valid_0's rmse: 12.9251
[700]	valid_0's rmse: 12.923
[800]	valid_0's rmse: 12.9198
[900]	valid_0's rmse: 12.9155
[1000]	valid_0's rmse: 12.9121
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 12.9121


<ipython-input-31-7801a0c50960>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-31-7801a0c50960>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.1913
[200]	valid_0's rmse: 12.9284
[300]	valid_0's rmse: 12.9013
[400]	valid_0's rmse: 12.8883
[500]	valid_0's rmse: 12.8826
[600]	valid_0's rmse: 12.8776
[700]	valid_0's rmse: 12.8742
[800]	valid_0's rmse: 12.8706
[900]	valid_0's rmse: 12.867
[1000]	valid_0's rmse: 12.8643
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 12.8643
